In [12]:
#export
from collections import OrderedDict
import json
import os
from operator import itemgetter
from multiprocessing import cpu_count
import numpy as np
import PIL.Image
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from catalyst.utils import get_one_hot


def list_files(folder):
    dirname = os.path.expanduser(folder)
    return [os.path.join(dirname, x) for x in os.listdir(dirname)]


def load_data(filenames=None):
    if filenames is None:
        filenames = [f'{fn}.json' for fn in ('train', 'test')]
    content = []
    for filename in filenames:
        with open(filename) as f:
            content.append(json.load(f))
    return content


class RxRxDataset(Dataset):
    
    def __init__(self, items, onehot=True,
                 label_smoothing=None,
                 open_fn=PIL.Image.open, tr=None):
        
        super().__init__()
        self.onehot = onehot
        self.label_smoothing = label_smoothing
        self.open_fn = open_fn
        self.tr = tr
        self.targets = [item['sirna'] for item in items]
        self.images = [item['images'] for item in items]
        self.num_classes = len(np.unique(self.targets))
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, index):
        bunch = self.images[index]
        channels = []
        for i, filename in sorted(bunch, key=itemgetter(0)):
            img = self.open_fn(filename)
            img = img if self.tr is None else self.tr(img)
            channels.append(img)
        sample = torch.cat(channels, dim=0)
        y = self.targets[index]
        sample = dict(features=sample, targets=y)
        if self.onehot:
            y_enc = get_one_hot(
                y, smoothing=self.label_smoothing, 
                num_classes=self.num_classes)
            sample['targets_one_hot'] = y_enc
        return sample
    

class _Split:
    
    def __init__(self, test_size=0.1, seed=1):
        self.test_size = test_size
        self.seed = seed
    
    def __call__(self, *args, **kwargs):
        raise NotImplementedError()
    
    
class TargetSplit(_Split):
    def __call__(self, records):
        ys = [r['sirna'] for r in records]
        train, valid = train_test_split(
            records, stratify=ys,
            test_size=self.test_size,
            random_state=self.seed)
        return train, valid
    

class DataBunch:
    
    def __init__(self, input_size=224, stats=None, aug_raw=None, aug_resized=None):
        self.input_size = input_size
        self.stats = stats or {'mean': (0.5,), 'std': (0.5,)}
        self.aug_raw = aug_raw
        self.aug_resized = aug_resized
        
    def create(self, train_records, test_records, **kwargs):
        trn_dl, val_dl = self.create_train_valid(train_records, **kwargs)
        tst_dl = self.create_test(test_records, **kwargs)
        loaders = OrderedDict([('train', trn_dl), ('valid', val_dl), ('test', tst_dl)])
        return loaders
        
    def create_train_valid(self, records, batch_size=4, split=None, test_size=0.1):
        split = split or TargetSplit(test_size)
        train, valid = split(records)
        trn_ds = RxRxDataset(train, tr=self.build_tr(True))
        val_ds = RxRxDataset(valid, tr=self.build_tr(False))
        trn_dl = self.build_dl(trn_ds, bs=batch_size)
        val_dl = self.build_dl(val_ds, bs=batch_size, shuffle=False)
        return trn_dl, val_dl
    
    def create_test(self, records, batch_size=4):
        tst_ds = RxRxDataset(records, tr=self.build_tr(False), onehot=False)
        tst_dl = self.build_dl(tst_ds, bs=batch_size, shuffle=False)
        return tst_dl
    
    def build_tr(self, train=True):
        tr = []
        if train:
            if self.aug_raw is not None:
                tr.extend(self.aug_raw)
        tr.append(T.Resize(self.input_size))
        if train:
            if self.aug_resized is not None:
                tr.extend(self.aug_resized)
        tr.append(T.ToTensor())
        tr.append(T.Normalize(self.stats['mean'], self.stats['std']))
        return T.Compose(tr)
    
    def build_dl(self, ds, bs, shuffle=True):
        return DataLoader(ds, shuffle=shuffle, batch_size=bs, num_workers=cpu_count())

In [15]:
!python -m jupytools export -nb "00d_data_bunch.ipynb" -o .

Exported: 00d_data_bunch.ipynb -> data_bunch.py
1 notebook(s) exported into folder: .
